In [4]:
import PyPDF2
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
pdf_paths = ["/home/bigdata/eva1_BDA/PDF/PIMENTON_MAYO.pdf", "/home/bigdata/eva1_BDA/PDF/recetas.pdf"] 

## Extraer info y dividirla

In [11]:
def extraer_info_pdf(pdf_path):
    #Extrae el texto de un archivo PDF usando PyPDF2.
    text_data = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text_data += page.extract_text() + " "
    except Exception as e:
        print(f"Se produjo un error al leer el PDF: {e}")
    
    return text_data


def dividir_texto(texto):
    #Divide el texto en fragmentos manejables para la indexación.
    #extrae el texto de los archivos PDF y lo divide en fragmentos de 500 caracteres, con un solapamiento de 50 caracteres,
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_text(texto)


def extraer_texto_y_dividir_pdf(pdf_path):
    #Extrae el texto de un archivo PDF y lo divide en fragmentos.
    texto = extraer_info_pdf(pdf_path)
    if not texto:
        print("El texto del PDF estaba vacío")
        texto = []
    
    return dividir_texto(texto)

def extraer_y_dividir_texto_de_pdfs(pdf_paths):
    #Une los dos pdf divididos
    all_chunks = []
    for pdf_path in pdf_paths:
        chunks = extraer_texto_y_dividir_pdf(pdf_path)
        all_chunks.extend(chunks)  # Unir todos los fragmentos de los diferentes PDFs
    return all_chunks

## Crear embeddings y vectorstore

In [17]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

chunks = extraer_y_dividir_texto_de_pdfs(pdf_paths)

vectorstore = Chroma.from_texts(
        texts=chunks,
        collection_name="pdf_data",
        embedding=embeddings,
        persist_directory="./vectorstoreCastellano"
    )

## Hacer consultas a Ollama

In [14]:
llm = OllamaLLM(model="llama3.2", server_url="http://localhost:11434")

# Función para realizar una consulta al vector store
def realizar_consulta(vectorstore, consulta):
    retriever = vectorstore.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    respuesta = qa_chain.run(consulta)
    return respuesta

In [19]:
# Ejemplo de consulta pdf PIMENTON_MAYO
consulta = "¿Resumeme como se hace el pulpo a la gallega?"
respuesta = realizar_consulta(vectorstore, consulta)
print(f"Respuesta: {respuesta}")

Respuesta: Claro, te resumiré cómo se hace el pulpo a la gallega:

1. Retira el pulpo del fuego y reserva.
2. Prepara las patatas picadas (en rodajas o trozos) y colócalas en el fondo del plato.
3. Corta los tentáculos en trozos de 1-2 cm y colócalos sobre las patatas.
4. Añade sal al gusto.
5. Mezcla pimentón picante y dulce en partes iguales y espolvorea por encima del pulpo.
6. Repite el proceso para los otros cuatro pulpos (en diferentes partes).
7. Una vez que todos los pulpos estén fritos, colócalos en el horno a 100°C hasta la hora de servir.
8. Prepara una salsa de ajo y cebolla pochada con coñac, luego tritura y cuela para obtener una salsa lista para usar.
9. Limpia y trocea las litiruelas para evitar que salpiquen en la sartén al freírlos.

Espero que esto te ayude a preparar un delicioso pulpo a la gallega.


In [18]:
# Ejemplo de consulta pdf recetas
consulta = "¿Resumeme como se hace el cordero asado?"
respuesta = realizar_consulta(vectorstore, consulta)
print(f"Respuesta: {respuesta}")

Respuesta: No tengo suficiente información para resumir exactamente cómo se hace el cordero asado. El texto proporcionado parece ser un paso a paso general sin mencionar específicamente la receta de cordero asado.

Sin embargo, puedo intentar resumir lo que se menciona en el texto:

* Se fritan las antenas del cordero en una sartén con aceite hasta que estén doradas.
* Se cocina el cordero en el horno a 100°C durante una hora y media.
* Se prepara una salsa con ajo, cebolla, coñac, vinagre de vino y agua.
* Se rocía la pierna del cordero con una mezcla de ajo machacado, aceite y vinagre de vino.

Espero que esto te sea útil. Si tienes alguna otra pregunta o necesitas más información, no dudes en preguntar.
